In [1]:
import alphalens

/usr/local/lib/python2.7/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Let's first read in the tickers

In [13]:
import pandas as pd
ticker = pd.read_csv('./Three+Factors+Alpha.csv', index_col = 0).index


In [14]:
ticker

Index([u'A', u'AA', u'AAL', u'AAP', u'AAPL', u'ABBV', u'ABC', u'ABMD', u'ABT',
       u'ACAD',
       ...
       u'YUM', u'YUMC', u'Z', u'ZAYO', u'ZBH', u'ZBRA', u'ZG', u'ZION',
       u'ZNGA', u'ZTS'],
      dtype='object', length=978)

now read in the price data

In [23]:
price = pd.DataFrame()
for s in ticker:
    price[s] = pd.read_csv('./price_data/'+s+'.csv', index_col =0, parse_dates=True).Close
    

create a dictionary that contains the ticker_sector

In [28]:
sector_name ={
    0: "xly",
    1: "xlp",
    2: "xle",
    3: "xlf",
    4: "xlv",
    5: "xli",
    6: "xlre",
    7: "xlk",
    8: "xlu",
    9: "xlb",
             }

In [30]:
sector ={}
for i in range(10):
    index = pd.read_csv('./rank_result/'+sector_name[i]+'_rank.csv', index_col =0).index
    for s in index:
        sector[s] = i
    

read in the factors